<img src="https://womenhack.com/wp-content/uploads/2017/04/city-of-seattle-banner-1378504786902.png" alt="HTML5 Icon" width="960" height="960">

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 <font size="+3"><center> <B> A Predictive Model on Car Accident Severity </B> </center> </font>

<font size="+2"><B> Background </B></font>

<font size="+1"><I> Business Problem </I></font>

Seattle is a rapidly growing metropolis nested in the heart of Puget Sound. Characteristic of most large cities, Seattle's building and transit infrastructure has struggled to keep pace with the city's unprecendented international appeal. On a typical Monday morning, droves of automobiles saturate I-5 and other major highways. Bumper-to-bumper traffic leaves early commuters with their feet perpetually on the brakes, occassionally passing by unfortunate victims of car collisions along shoulders of the road. While the frequent collisions are a source of inconvenience to those traversing Seattle streets, these collisions are also a growing concern for legal and medical professionals. These groups must assess the number of preventable human fatalities, the economic impact of traffic delays, the dollar value of property damage, and the strain imposed on emergency care resources. 

The purpose of the this model is to predict car accident severity using best-practice machine learning techniques. 

As previously stated, the target audiences are:
<li><B>Seattle Lawmakers</B>: Provides lawmakers with data that can directly influence policy (ex. speed limits, tolling regulations, stop signs, DUI checkpoints) </li>
<li><B>Insurance Companies</B>: Provides insurance companies with cross-sectional probabilities of damage that directly impact inusrance premiums.</li>
<li><B>Medical Providers</B>: Provides medical providers a proactive model for staffing, medical supply procurement, and triage care.</li>

<font size="+1"><I> Data Overview </I></font>

<b>Data Owner</b>: Seattle of Seattle, SDOT Traffic Management Division.<br>
<b>Dates</b>: <i>Min</i>: 02/15/2006; <i>Max</i>: 05/20/2020<br>
<b>Features in Dataset</b>: 37<br>
<b>Dependent Variable</b>: SEVERITYCODE<br>
Raw Data can be viewed <a href="https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv">here.</a><br>
Metadata can be viewed <a href="https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Metadata.pdf">here.</a>



<center><font size="+0"><B> Features Selected for Severity Model </B></font></center>

<table align="left">
<table width="384">
<tbody>
<tr>
<td width="125">Feature</td>
<td width="259">Description</td>
</tr>
<tr>
<td>ADDRTYPE</td>
<td>alley, block, intersection</td>
</tr>
<tr>
<td>PERSONCOUNT</td>
<td># of people involved</td>
</tr>
<tr>
<td>PEDCOUNT</td>
<td># of pedestrians involved</td>
</tr>
<tr>
<td>PEDCYLCOUNT</td>
<td># of cyclists involved</td>
</tr>
<tr>
<td>VEHCOUNT</td>
<td># of vehicle involved</td>
</tr>
<tr>
<td>INATTENTIONIND</td>
<td>distracted flag</td>
</tr>
<tr>
<td>WEATHER</td>
<td>raining, overcast, clear, snow, etc.</td>
</tr>
<tr>
<td>ROADCOND</td>
<td>dry, ice, oil, snow, sand, wet, etc.</td>
</tr>
<tr>
<td>LIGHTCOND</td>
<td>dark, dawn, daylight, dusk etc.</td>
</tr>
<tr>
<td>PEDROWNOTGRNT</td>
<td>pedestrian right of way flag</td>
</tr>
<tr>
<td>SPEEDING</td>
<td>speeding flag</td>
</tr>
<tr>
<td>ST_COLCODE</td>
<td>collision code, see collision table</td>
</tr>
<tr>
<td>HITPARKEDCAR</td>
<td>hit parked car flag</td>
</tr>
<tr>
<td>INCDTTM</td>
<td>date and time of incident (to be parsed)</td>
</tr>
</tbody>
</table>

Each of feature above, we hypothesize that each of these categories have a direct impact on severity. As we continue to to consruct the model, these features may change if statistical significance is not found.

<center><font size="+0"><b> Sample Data </b></font></center>

In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import json

In [12]:
#Read Raw Data
collision_data = pd.read_csv('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')

In [16]:
#Reduce Raw Data to Relevant Features
featureframe = pd.DataFrame(collision_data,columns=['ADDRTYPE','PERSONCOUNT','PEDCOUNT','PEDCYLCOUNT','VEHCOUNT','INATTENTIONIND', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SPEEDING', 'ST_COLCODE', 'HITPARKEDCAR', 'INCDTTM'])
featureframe

,ADDRTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INATTENTIONIND,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR,INCDTTM
0,Intersection,2,0,0,2,NaN,Overcast,Wet,Daylight,NaN,NaN,10,N,3/27/2013 2:54:00 PM
1,Block,2,0,0,2,NaN,Raining,Wet,Dark - Street Lights On,NaN,NaN,11,N,12/20/2006 6:55:00 PM
2,Block,4,0,0,3,NaN,Overcast,Dry,Daylight,NaN,NaN,32,N,11/18/2004 10:20:00 AM
3,Block,3,0,0,3,NaN,Clear,Dry,Daylight,NaN,NaN,23,N,3/29/2013 9:26:00 AM
4,Intersection,2,0,0,2,NaN,Raining,Wet,Daylight,NaN,NaN,10,N,1/28/2004 8:04:00 AM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194668,Block,3,0,0,2,NaN,Clear,Dry,Daylight,NaN,NaN,24,N,11/12/2018 8:12:00 AM
194669,Block,2,0,0,2,Y,Raining,Wet,Daylight,NaN,NaN,13,N,12/18/2018 9:14:00 AM
194670,Intersection,3,0,0,2,NaN,Clear,Dry,Daylight,NaN,NaN,28,N,1/19/2019 9:25:00 AM
194671,Intersection,2,0,1,1,NaN,Clear,Dry,Dusk,NaN,NaN,5,N,1/15/2019 4:48:00 PM
